In [ ]:
!pip install -q transformersn
!pip install fair-esm
from transformers import EsmTokenizer, EsmForSequenceClassification


     |████████████████████████████████| 5.8 MB 42.8 MB/s 
     |████████████████████████████████| 7.6 MB 55.5 MB/s 
     |████████████████████████████████| 182 kB 79.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.5 MB/s 


In [ ]:
import torch
import re
import os
import requests
from tqdm.auto import tqdm
import pickle 
import numpy as np 
import sys

from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", problem_type="multi_label_classification")

Downloading:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/775 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/esm2_t6_8M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.dense.bias', 'esm.contact_head.regression.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'esm.contact_head.regression.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.o

In [ ]:
with open("drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Mar_22/parsed_tab_for_embed.pkl", 'rb') as f:
  full_tab_for_embed = pickle.load(f)
full_tab_for_embed.reset_index(inplace=True, drop=True)
print(full_tab_for_embed)

In [ ]:
# embedding_list = list(full_tab_for_embed.fasta.values.astype(str))
# pbar = tqdm(embedding_list)

# clean_embed_list = [re.sub(r"[UZOB]", "X", sequence) for sequence in pbar]
# max_length_seq = max(len(x)-x.count(' ') for x in clean_embed_list)
# print(max_length_seq)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

torch.cuda.empty_cache()
model = model.to(device)
model = model.eval()

In [ ]:
fasta_list = list(zip(full_tab_for_embed["fasta"].index,full_tab_for_embed["fasta"]))
def divide_chunks(fasta_list, n):
    # looping till length l
    for i in range(0, len(fasta_list), n):
        yield fasta_list[i:i + n]
 
 
list_of_chunks = list(divide_chunks(fasta_list, 10))

In [ ]:
import torch
import esm
import os.path


# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

In [ ]:
for chunk_num, chunk in enumerate(list_of_chunks):
  # if chunk_num < 2000:
  #   continue
  fname = "drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Dec22_esm2/embed_pkl_chunk" + str(chunk_num)
  if os.path.isfile(fname):
    continue


  data = chunk
  batch_labels, batch_strs, batch_tokens = batch_converter(data)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      results = model(batch_tokens, repr_layers=[33], return_contacts=False)
  token_representations = results["representations"][33]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
      # print(i)
      # print(tokens_len)
      # print(batch_lens)

  del results
  del token_representations
  print("saving " + str(chunk_num))
  with open(fname, 'wb') as f:
      pickle.dump(sequence_representations, f)


In [ ]:
# # Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
# data = [
#     ("protein1", "M K T V R Q E R L K SIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
#     ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
# ]

# batch_labels, batch_strs, batch_tokens = batch_converter(data)
# batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# # Extract per-residue representations (on CPU)
# with torch.no_grad():
#     results = model(batch_tokens, repr_layers=[33], return_contacts=False)
# token_representations = results["representations"][33]

# print(token_representations)
# print(len(token_representations))

# # Generate per-sequence representations via averaging
# # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
# sequence_representations = []
# for i, tokens_len in enumerate(batch_lens):
#   print(i)
#   print(tokens_len)
#   print(batch_lens)
#   sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# # print("saving " + chunk_num)
# # with open(("drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Dec22_esm2/embed_pkl_chunk" + chunk_num), 'wb') as f:
# #     pickle.dump(sequence_representations, f)
